In [2]:
# Load data
import pandas as pd
data=pd.read_csv('C:/Users/kk/Desktop/data/data_berka/trans.asc',sep=';')
data=data.drop(['trans_id','account'],axis=1) # delete redundant features
data=data.dropna(how='any')
 
# identify categorical and continous features 
cat=['account_id','date','type','operation','k_symbol','bank']
conti=['amount','balance']
 
# fit the OrdinalEncoder on entire dataset 
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
oe.fit(data[cat])
 
# split data into training and testing set
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.3)
 
# convert training categorical features to numeric 
train_cat=oe.transform(train[cat])
# concate categorical features and continous features for training set
import numpy as np
train_oe=np.append(train_cat,train[conti],axis=1)
 
# apply onehotencoder on train_array
from sklearn.preprocessing import OneHotEncoder
# fit onehotencoder only on training set
cat_index=[i for i in range(len(cat))]
ohe=OneHotEncoder(handle_unknown='ignore',categorical_features=cat_index)
train_ohe=ohe.fit_transform(train_oe)
train_ohe

D:\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


<161325x4197 sparse matrix of type '<class 'numpy.float64'>'
	with 1290600 stored elements in COOrdinate format>

In [35]:
class new_OrdinalEncoder():
    def __init__(self,cat_index='all'):
        self.dicts={}
        # cate_index is the categorical feature index list
        self.cat_index=cat_index
        
    def fit(self,df):
        if self.cat_index=='all':
            self.cat_index=list(range(df.shape[1]))
        for feat in self.cat_index:
            dic=np.unique(df.iloc[:,feat])
            dic=dict([(i,index) for index, i in enumerate(dic)])
            self.dicts[feat]=dic
            
    def fit_transform(self,df):
        if self.cat_index=='all':
            self.cat_index=list(range(df.shape[1]))
        df_output=df.copy()
        for feat in self.cat_index:
            dic=np.unique(df.iloc[:,feat])
            dic=dict([(i,index) for index, i in enumerate(dic)])
            self.dicts[feat]=dic
            df_output.iloc[:,feat]=df.iloc[:,feat].apply(lambda x: dic[x])
        return df_output
        
    def transform(self,df):
        df_output=df.copy()
        for feat in self.cat_index:
            dic=self.dicts[feat]
            df_output.iloc[:,feat]=df.iloc[:,feat].apply(self.unknown_value,args=(dic,))
        return df_output
    
    def unknown_value(self,value,dic): # It will set up a new interger for unknown values!
        try:
            return dic[value]
        except:
            return len(dic)

In [37]:
cat_index=[0,1,2,3,6,7]
new_oe=new_OrdinalEncoder(cat_index)
train_oe=new_oe.fit_transform(train)
ohe=OneHotEncoder(handle_unknown='ignore',categorical_features=cat_index)
train_ohe=ohe.fit_transform(train_oe,cat_index)
train_ohe

D:\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


<161325x4197 sparse matrix of type '<class 'numpy.float64'>'
	with 1290600 stored elements in COOrdinate format>

In [38]:
test_oe=new_oe.transform(test)
test_ohe=ohe.transform(test_oe)

In [34]:
dict([(1,3)])

{1: 3}

In [52]:
noe=new_OrdinalEncoder(cat_index)
noe.fit(train)

In [41]:
from sklearn.pipeline import Pipeline
cat_index=[0,1,2,3,6,7]
estimators=[('new_ordianlencoder',new_OrdinalEncoder(cat_index)),
            ('onehotencoder',OneHotEncoder(handle_unknown='ignore',categorical_features=cat_index))]
pipe=Pipeline(estimators)
pipe

Pipeline(memory=None,
     steps=[('new_ordianlencoder', <__main__.new_OrdinalEncoder object at 0x00000211B8F015F8>), ('onehotencoder', OneHotEncoder(categorical_features=[0, 1, 2, 3, 6, 7], categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True))])

In [59]:
ohe.transform(noe.transform(test))

<69140x4197 sparse matrix of type '<class 'numpy.float64'>'
	with 553111 stored elements in COOrdinate format>

In [89]:
############################

In [90]:
fit_params_steps

{'ohe': {}, 'oe': {}}

In [67]:
from itertools import islice
def _iter(steps,with_final=True, filter_passthrough=True):
        """
        Generate (idx, (name, trans)) tuples from self.steps
        When filter_passthrough is True, 'passthrough' and None transformers
        are filtered out.
        """
        stop = len(steps)
        if not with_final:
            stop -= 1

        for idx, (name, trans) in enumerate(islice(steps, 0, stop)):
            if not filter_passthrough:
                yield idx, name, trans
            elif trans is not None and trans != 'passthrough':
                yield idx, name, trans

In [72]:
def _fit_transform_one(transformer,
                       X,
                       y,
                       weight,
                       message_clsname='',
                       message=None,
                       **fit_params):
    """
    Fits ``transformer`` to ``X`` and ``y``. The transformed result is returned
    with the fitted transformer. If ``weight`` is not ``None``, the result will
    be multiplied by ``weight``.
    """
    with _print_elapsed_time(message_clsname, message):
        if hasattr(transformer, 'fit_transform'):
            res = transformer.fit_transform(X, y, **fit_params)
        else:
            res = transformer.fit(X, y, **fit_params).transform(X)

    if weight is None:
        return res, transformer
    return res * weight, transformer

In [80]:
from sklearn.utils.validation import check_memory
memory=check_memory(None)
fit_transform_one_cached = memory.cache(_fit_transform_one)
fit_params_steps = {name: {} for name, step in steps
                            if step is not None}
print(memory)
print(fit_params_steps)

Memory(location=None)
{'ohe': {}, 'oe': {}}


In [108]:
import sklearn
sklearn.__version__

'0.20.3'

In [99]:
step_idx=0
def _log_message(step_idx,verbose=False):
        if not verbose:
            return None
        name, step = steps[step_idx]

        return '(step %d of %d) Processing %s' % (step_idx + 1,
                                                  len(steps),
                                                  name)
_log_message(step_idx)

In [109]:
from .utils import Bunch, _print_elapsed_time
fit_transform_one_cached(
                    cloned_transformer, Xt, None, None,
                    message_clsname='Pipeline',
                    message=None,
                    **{})

ModuleNotFoundError: No module named '__main__.utils'; '__main__' is not a package

In [101]:
Xt=train
for (step_idx, name, transformer) in _iter(steps,with_final=False,filter_passthrough=False):
    print((step_idx, name, transformer))
    if hasattr(memory, 'location'):
                # joblib >= 0.12
                if memory.location is None:
                    # we do not clone when caching is disabled to
                    # preserve backward compatibility
                    cloned_transformer = transformer
                else:
                    cloned_transformer = clone(transformer)
                    
    Xt, fitted_transformer = fit_transform_one_cached(
                    cloned_transformer, Xt, None, None,
                    message_clsname='Pipeline',
                    message=None,
                    **fit_params_steps[name])

(0, 'ohe', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True))


NameError: name '_print_elapsed_time' is not defined

In [92]:
Xt=train
fit_params={}


In [ ]:
#######################

In [63]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
steps=[('ohe',OneHotEncoder()),('oe',OrdinalEncoder())]
names,estimators=zip(*steps)
transformers=estimators[:-1]
estimator=estimators[-1]

('ohe', 'oe')

In [94]:
pipe.fit(train)

TypeError: fit_transform() takes 2 positional arguments but 3 were given